## Imports

In [26]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as T
import bcolz
import time
from torch.utils.data import Dataset, DataLoader, sampler

%matplotlib inline

In [27]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

def gpu(x,use_gpu=use_gpu):
    if use_gpu:
        return x.cuda()
    else:
        return x

Using gpu: True 


## Data processing

In [28]:
data_dir = './Dataset'

transform = T.Compose([
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dsets = {x: dset.ImageFolder(os.path.join(data_dir, x), transform=transform)
         for x in ['train', 'val', 'test']}

train_dataset = dset.ImageFolder('./Dataset/train', transform=transform)
val_dataset = dset.ImageFolder('./Dataset/val', transform=transform)
test_dataset = dset.ImageFolder('./Dataset/test', transform=transform)

In [29]:
train_size = len(train_dataset)
val_size = len(val_dataset)
test_size = len(test_dataset)
print("Number of training examples {}, validation examples {}, testing examples {}".format(train_size, val_size, test_size))

Number of training examples 66071, validation examples 11016, testing examples 33154


In [30]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=6)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=6)

## Creating VGG model

In [31]:
model_vgg = models.vgg16(pretrained=True)
print(model_vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [32]:
for param in model_vgg.parameters():
    param.requires_grad = False
model_vgg.classifier._modules['6'] = nn.Linear(4096, 172)
model_vgg.classifier._modules['7'] = torch.nn.LogSoftmax(dim = 1)

In [33]:
print(model_vgg.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=172, bias=True)
  (7): LogSoftmax(dim=1)
)


In [34]:
model_vgg = gpu(model_vgg)

### Calculating preconvoluted features

In [38]:
def preconvfeat_val(dataset):
    ''' Calculates the outputs of model_vgg.features (i.e., before the FC layers) on the validation dataset.
        Returns a list of the features generated for each image, and a list of their corresponding labels.'''
    conv_features = []
    labels_list = []
    i = 0 # Counter for how many batches we have processed so far.
    for data in dataset:
        if i % 10 == 0:
            print("Completed {} forwarded passes".format(i))
        inputs,labels = data
        inputs = gpu(inputs)
        labels = gpu(labels)
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
        i += 1
    conv_features = np.concatenate([[feat] for feat in conv_features])
    return (conv_features,labels_list)

In [39]:
def preconvfeat_train(dataset, batches_per_file=100):
    ''' Calculates the outputs of model_vgg.features (i.e., before the FC layers) on the training dataset.
        Returns a list of the features generated for each image, and a list of their corresponding labels.'''
    output_conv_features = [] # Contains a list of lists features. Each sub-list will be saved in its own file.
    conv_features = []
    output_labels_list = []
    labels_list = []
    i = 0 # Counter for how many batches we have processed so far.
    for data in dataset:
        # Once we have made 'batches_per_file' forward passes, we concatenate the features for each of those 
        # forward passes together, and add them to output_conv_features. This is done mainly to limit the size of 
        # the number of features we're concatenating together for the large training dataset.
        if i >= batches_per_file:
            print("Completed {} forward passes".format(batches_per_file))
            i = 0
            output_conv_features.append(np.concatenate([[feat] for feat in conv_features]))
            output_labels_list.append(labels_list)
            conv_features = []
            labels_list = []
        inputs,labels = data
        inputs = gpu(inputs)
        labels = gpu(labels)
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
        i += 1
    if len(conv_features) > 0:
        output_conv_features.append(np.concatenate([[feat] for feat in conv_features]))
        output_labels_list.append(labels_list)
    return (output_conv_features, output_labels_list)

In [37]:
%%time
conv_feat_train,labels_train = preconvfeat_train(train_dataloader)

Completed 0 forwarded passes
Completed 10 forwarded passes
Completed 20 forwarded passes
Completed 30 forwarded passes
Completed 40 forwarded passes
Completed 50 forwarded passes
Completed 60 forwarded passes
Completed 70 forwarded passes
Completed 80 forwarded passes
Completed 90 forwarded passes
Completed 100 forwarded passes
Completed 10 forwarded passes
Completed 20 forwarded passes
Completed 30 forwarded passes
Completed 40 forwarded passes
Completed 50 forwarded passes
Completed 60 forwarded passes
Completed 70 forwarded passes
Completed 80 forwarded passes
Completed 90 forwarded passes
Completed 100 forwarded passes
Completed 10 forwarded passes
Completed 20 forwarded passes
Completed 30 forwarded passes
Completed 40 forwarded passes
Completed 50 forwarded passes
Completed 60 forwarded passes
Completed 70 forwarded passes
Completed 80 forwarded passes
Completed 90 forwarded passes
Completed 100 forwarded passes
Completed 10 forwarded passes
Completed 20 forwarded passes
Complete

In [40]:
%%time
conv_feat_val,labels_val = preconvfeat_val(val_dataloader)

Completed 0 forwarded passes
Completed 10 forwarded passes
Completed 20 forwarded passes
Completed 30 forwarded passes
Completed 40 forwarded passes
Completed 50 forwarded passes
Completed 60 forwarded passes
Completed 70 forwarded passes
Completed 80 forwarded passes
Completed 90 forwarded passes
Completed 100 forwarded passes
Completed 110 forwarded passes
Completed 120 forwarded passes
Completed 130 forwarded passes
Completed 140 forwarded passes
Completed 150 forwarded passes
Completed 160 forwarded passes
Completed 170 forwarded passes
CPU times: user 1min 13s, sys: 26.2 s, total: 1min 39s
Wall time: 1min 42s


In [21]:
def save_array_train(fname, arr):
    for i, arr_list in enumerate(arr): 
        c=bcolz.carray(arr_list, rootdir=fname + "_" + str(i) + '.bc', mode='w')
    c.flush()

def save_array_val(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

%mkdir ./vgg16

mkdir: cannot create directory ‘./vgg16’: File exists


In [ ]:
save_array_train(os.path.join('./vgg16','feat_train'), conv_feat_train)
save_array_train(os.path.join('./vgg16','labels_train'), labels_train)
save_array_val(os.path.join('./vgg16','feat_val.bc'), conv_feat_val)
save_array_val(os.path.join('./vgg16','labels_val.bc'),labels_val)

## Training fully connected module

### Loading Preconvoluted features

In [27]:
conv_feat_val = load_array('./vgg16/feat_val.bc')
labels_val = load_array('./vgg16/labels_val.bc')

In [28]:
print(len(conv_feat_val))
print(conv_feat_val.shape)

11016
(11016, 512, 7, 7)
